In [8]:
import pandas as pd

In [9]:
df2 = pd.read_csv("C:\\Users\\Gebruiker\\Documents\\PycharmProjects\\PythonProject\\merged_features_IC50_g12d.csv")
df2

In [ ]:
import pandas as pd
df_NN_molecules_Newfeatures_G12C = pd.read_csv('..\..\..\..\..\AppData\Local\Packages\5319275A.WhatsAppDesktop_cv1g1gvanyjgm\TempState\885B09B2268BB141018CE1D96FE82A7E\NN_molecules_Newfeatures_G12C.csv')
df_NN_molecules_Newfeatures_G12C

FileNotFoundError: [Errno 2] No such file or directory: '../../../../../AppData/Local/Packages/5319275A.WhatsAppDesktop_cv1g1gvanyjgm/TempState/885B09B2268BB141018CE1D96FE82A7E/NN_molecules_Newfeatures_G12C.csv'

In [10]:
import numpy as np
import pandas as pd
def pIC50(input):
    pIC50 = []

    input["IC50 (nM)"] = pd.to_numeric(input["IC50 (nM)"],errors='coerce')

    for i in input["IC50 (nM)"]:
        molar = i*(10**-9) # Converts nM to M
        pIC50.append(-np.log10(molar))

    input['pIC50'] = pIC50
    x = input["pIC50"]

    return x

In [12]:
filename = "C:\\Users\\Gebruiker\\Documents\\PycharmProjects\\PythonProject\\merged_features_IC50_g12d.csv"
#df = pd.read_csv("fda_original.csv",sep=";")
df2 = pd.read_csv(filename)
df2.dropna(subset=['IC50 (nM)'], inplace=True)
df2['IC50 (nM)'] = pIC50(df2)
df2['IC50 (nM)'] = df2['pIC50']  # Replace 'SV' column with pIC50 values

# Drop 'pIC50' column (optional) as it's now redundant
df2.drop(columns=['pIC50'], inplace=True)

df2

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Load training descriptors data
#df1 = df1.loc[:, ~df1.columns.str.contains('Unnamed')]
df2.dropna(subset=['IC50 (nM)'], inplace=True)
X = df2.drop(columns=['IC50 (nM)','Smiles', 'ChEMBL ID'])
y = df2['IC50 (nM)']

# Initialize model and pipeline
model = RandomForestRegressor(max_depth=20, min_samples_leaf=10)
pipe = Pipeline([('scaler', StandardScaler()), ('model', model)])

# Load new descriptors data
new_descriptors = pd.read_csv('C:\\Users\\Gebruiker\\Documents\\PycharmProjects\\PythonProject\\FDA_Hyb_Features.csv')
new_descriptors.dropna(inplace=True)

# Extract ChEMBL IDs and SMILES
chembl_id_column = new_descriptors['ChEMBL ID']
smiles_column = new_descriptors['Smiles']
new_descriptors.drop(columns=['ChEMBL ID', 'Smiles'], inplace=True)

# Ensure new_descriptors has the same columns as X
missing_cols = set(X.columns) - set(new_descriptors.columns)  # Columns missing in new data
extra_cols = set(new_descriptors.columns) - set(X.columns)  # Extra columns in new data

# Add missing columns with zero values
for col in missing_cols:
    new_descriptors[col] = 0

# Drop extra columns
new_descriptors = new_descriptors[X.columns]  # Reorder and drop extras

# Now X and new_descriptors have identical columns

# Initialize a DataFrame to store results
loop_results_df = pd.DataFrame(columns=['ChEMBL ID', 'Smiles', 'Predicted Value'])

# Number of iterations
loop_n = 50

# Dictionary to store predicted values for each ChEMBL ID
predicted_values = {chembl_id: [] for chembl_id in chembl_id_column.unique()}

# Run the loop to fit the model and predict the values
for i in range(loop_n):
    pipe.fit(X, y)

    loop_predicted_values = pipe.predict(new_descriptors)

    loop_df = pd.DataFrame({
        'ChEMBL ID': chembl_id_column,
        'Smiles': smiles_column,
        'Predicted Value': loop_predicted_values
    })
    loop_df.sort_values(by='Predicted Value', ascending=False, inplace=True)

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning)
        loop_results_df = pd.concat([loop_results_df, loop_df.head(10)])

    # Store the predicted values for each ChEMBL ID
    for chembl_id, predicted_value in zip(chembl_id_column, loop_predicted_values):
        if chembl_id in predicted_values:
            predicted_values[chembl_id].append(predicted_value)

# Calculate frequency of each ChEMBL ID in the top 10
value_counts = loop_results_df['ChEMBL ID'].value_counts(normalize=True) * 10
#print(value_counts)

# Get the top 13 ChEMBL IDs
top_10_chembl_ids = value_counts.head(10).index

In [14]:
mean_values = {chembl_id: np.mean(predicted_values[chembl_id]) for chembl_id in top_10_chembl_ids}
std_dev_values = {chembl_id: np.std(predicted_values[chembl_id]) for chembl_id in top_10_chembl_ids}

summary_df = pd.DataFrame({
    'ChEMBL ID': top_10_chembl_ids,
    'Frequency': [value_counts[chembl_id] for chembl_id in top_10_chembl_ids],
    'Avg Predicted Value': [mean_values[chembl_id] for chembl_id in top_10_chembl_ids]
})

summary_df.head(10)
summary_df.to_csv("RF_molecules_Newfeatures_G12D.csv", index=False)